# Fetching Zip Codes with USPS Api

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

import requests
import pandas as pd
from xml.etree import ElementTree as ET
from tqdm.auto import trange
import time


def zip_lookup_old(addresses, usps_api_key, city='ALEXANDRIA', state='VA'):
    # Start the XML request body
    xml = f'<ZipCodeLookupRequest USERID="{usps_api_key}">'
    
    # Add each address to the request
    for address in addresses:
        xml += f'''
        <Address ID="{address['id']}">
            <Address1></Address1>
            <Address2>{address['A_Properties']}</Address2>
            <City>{city}</City>
            <State>{state}</State>
            <Zip5></Zip5>
            <Zip4></Zip4>
        </Address>'''

    # Close the XML request body
    xml += '</ZipCodeLookupRequest>'

    # Prepare the request parameters
    params = {
        'API': 'ZipCodeLookup',
        'XML': xml
    }

    # Send the request
    response = requests.get('http://production.shippingapis.com/ShippingAPI.dll', params=params)
    return response.text

def zip_lookup(addresses, usps_api_key, city='ALEXANDRIA', state='VA', max_retries=3, retry_delay=5, request_timeout=10):
    # Start the XML request body
    xml = f'<ZipCodeLookupRequest USERID="{usps_api_key}">'
    
    # Add each address to the request
    for address in addresses:
        xml += f'''
        <Address ID="{address['id']}">
            <Address1></Address1>
            <Address2>{address['A_Properties']}</Address2>
            <City>{city}</City>
            <State>{state}</State>
            <Zip5></Zip5>
            <Zip4></Zip4>
        </Address>'''

    # Close the XML request body
    xml += '</ZipCodeLookupRequest>'

    # Prepare the request parameters
    params = {
        'API': 'ZipCodeLookup',
        'XML': xml
    }

    attempts = 0
    while attempts < max_retries:
        try:
            # Send the request with a timeout
            response = requests.get('http://production.shippingapis.com/ShippingAPI.dll', params=params, timeout=request_timeout)
            response.raise_for_status()  # Raise an error for bad status codes
            return response.text
        except requests.RequestException as e:
            logging.warning(f"Request failed (attempt {attempts+1}/{max_retries}): {e}")
            attempts += 1
            time.sleep(retry_delay)  # Wait before retrying

    # If all retries fail, raise an exception
    raise Exception(f"All {max_retries} retries failed for the USPS API request")

def process_addresses(df, usps_api_key):
    # Add an ID column for tracking purposes
    df['id'] = df.index
    # ID_COL = 'Num'

    # Prepare a dictionary to store the results
    results = {}

    # Process the DataFrame in chunks of 5
    for start in trange(0, len(df), 5):
        chunk = df.iloc[start:start+5]
        addresses = chunk.to_dict('records')
        response = zip_lookup(addresses, usps_api_key)
        # print(f"Response: `{response}`")
        # Parse the response and update results
        tree = ET.ElementTree(ET.fromstring(response))

        for addr in tree.iter('Address'):
            id_ = addr.get('ID')
            zip5 = addr.find('Zip5').text if addr.find('Zip5') is not None else None
            zip4 = addr.find('Zip4').text if addr.find('Zip4') is not None else None
            logging.info(f"Address ID: {id_} \tZip5: {zip5} \tZip4: {zip4}") 
            # print(f"Zip5: {zip5}, Zip4: {zip4}")
            results[int(id_)] = {'zip5': zip5, 'zip4': zip4}

    # Map the results back to the DataFrame
    for id_, zip_info in results.items():
        df.loc[id_, 'Zip5'] = zip_info['zip5']
        df.loc[id_, 'Zip4'] = zip_info['zip4']

    return df


C:\Users\happy\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("../data/Alexandria_Tax_Info.csv", index_col="Num")

In [3]:
usps_api_key = '67547COLLEP53'
updated_df = process_addresses(data, usps_api_key)

  0%|                                                                                         | 0/7893 [00:00<?, ?it/s]INFO:root:Address ID: 50299410 	Zip5: None 	Zip4: None
INFO:root:Address ID: 50299400 	Zip5: 22302 	Zip4: 1308
INFO:root:Address ID: 50466800 	Zip5: 22302 	Zip4: 1324
INFO:root:Address ID: 50466810 	Zip5: 22302 	Zip4: 1324
INFO:root:Address ID: 50466820 	Zip5: 22302 	Zip4: 1324
  0%|                                                                                 | 1/7893 [00:00<36:49,  3.57it/s]INFO:root:Address ID: 50466830 	Zip5: 22302 	Zip4: 1324
INFO:root:Address ID: 50466840 	Zip5: 22302 	Zip4: 1324
INFO:root:Address ID: 50466850 	Zip5: 22302 	Zip4: 1324
INFO:root:Address ID: 50466860 	Zip5: 22302 	Zip4: 1324
INFO:root:Address ID: 50466870 	Zip5: 22302 	Zip4: 1324
  0%|                                                                                 | 2/7893 [00:00<31:03,  4.23it/s]INFO:root:Address ID: 50466880 	Zip5: 22302 	Zip4: 1324
INFO:root:Address ID: 504668

KeyboardInterrupt: 

In [ ]:
updated_df

In [15]:
updated_df['Zip5'].info()

<class 'pandas.core.series.Series'>
Index: 39461 entries, 50299410 to 26312030
Series name: Zip5
Non-Null Count  Dtype 
--------------  ----- 
36466 non-null  object
dtypes: object(1)
memory usage: 1.6+ MB


In [16]:
updated_df.to_csv("zip_codes.csv")

## Clean DF

In [4]:
updated_df = pd.read_csv("zip_codes.csv")
updated_df

,Num,Unnamed: 0,A_Properties,A_SQFT,id,Zip5,Zip4
0,50299410,0,3000 S 28TH ST,1,50299410,NaN,NaN
1,50299400,1,3102 S 28TH ST,1,50299400,22302.0,1308.0
2,50466800,2,"3200 S 28TH ST, UNIT 101",1090,50466800,22302.0,1324.0
3,50466810,3,"3200 S 28TH ST, UNIT 102",1280,50466810,22302.0,1324.0
4,50466820,4,"3200 S 28TH ST, UNIT 201",700,50466820,22302.0,1324.0
...,...,...,...,...,...,...,...
39456,50469430,39456,400 YOAKUM PY,0,50469430,NaN,NaN
39457,50469440,39457,450 YOAKUM PY,1,50469440,NaN,NaN
39458,26312010,39458,3630 ZABRISKIE DR,1,26312010,22304.0,1753.0
39459,26312020,39459,3634 ZABRISKIE DR,1,26312020,22304.0,1753.0


In [5]:
updated_df.drop(columns=['Unnamed: 0', 'id'])

,Num,A_Properties,A_SQFT,Zip5,Zip4
0,50299410,3000 S 28TH ST,1,NaN,NaN
1,50299400,3102 S 28TH ST,1,22302.0,1308.0
2,50466800,"3200 S 28TH ST, UNIT 101",1090,22302.0,1324.0
3,50466810,"3200 S 28TH ST, UNIT 102",1280,22302.0,1324.0
4,50466820,"3200 S 28TH ST, UNIT 201",700,22302.0,1324.0
...,...,...,...,...,...
39456,50469430,400 YOAKUM PY,0,NaN,NaN
39457,50469440,450 YOAKUM PY,1,NaN,NaN
39458,26312010,3630 ZABRISKIE DR,1,22304.0,1753.0
39459,26312020,3634 ZABRISKIE DR,1,22304.0,1753.0


In [6]:
tax = pd.read_csv("../data/Alexandria_Property_Holders.csv")
tax


,Num,year,A_Annual_Assesement,A_Taxes_and_Fees
0,50299400,2023,4395607.0,2410.80
1,50299400,2022,4395607.0,2296.00
2,50299400,2021,4395607.0,1960.00
3,50299400,2020,4395607.0,1120.00
4,50299400,2019,4395607.0,1120.00
...,...,...,...,...
484572,50469430,2015,56650.0,590.86
484573,50469430,2014,56650.0,590.86
484574,50469430,2013,56650.0,588.03
484575,50469430,2012,56650.0,565.37


In [7]:
combined_df = pd.merge(tax, updated_df[['Num', 'Zip5']], on='Num', how='inner')
cleaned = combined_df.dropna()
cleaned

,Num,year,A_Annual_Assesement,A_Taxes_and_Fees,Zip5
0,50299400,2023,4395607.0,2410.80,22302.0
1,50299400,2022,4395607.0,2296.00,22302.0
2,50299400,2021,4395607.0,1960.00,22302.0
3,50299400,2020,4395607.0,1120.00,22302.0
4,50299400,2019,4395607.0,1120.00,22302.0
...,...,...,...,...,...
466601,17341500,2015,1064188.0,11436.48,22314.0
466602,17341500,2014,999612.0,10758.45,22314.0
466603,17341500,2013,979751.0,10501.82,22314.0
466604,17341500,2012,966835.0,9321.01,22314.0


In [8]:
cleaned.to_csv("Cleaned_Data.csv")